In [6]:
import TensorCrossInterpolation as TCI

grid = 100

function f(v)
    return sin(2*π*v/grid)
end

function mcmc_sum(step)
    sumSgn = 0.0
    
    j::Int = 50
    jump::Int = 0

    now = f(j)
    sumSgn += sign(now)

    for _ in 1:(step-1)
        k = round(Int, grid/100)
        jump = rand(-k:k)

        if (j + jump) < 0 || (j + jump) > grid
            sumSgn += sign(now)
        else
            j = j + jump
            prp = f(j)

            accRatio = abs(prp/now)
            if rand() < accRatio
                now = prp
                sumSgn += sign(now)
            else
                sumSgn += sign(now) 
                j = j - jump # undo
            end
        end

    end

    return sumSgn / step
end

function exact_sum(grid)
    extVal = 0.0
    for v = 0:grid
        extVal += sin(2*π*v/grid)
    end
    return extVal / grid
end

function exact_sum_abs(grid)
    extVal = 0.0
    for v = 0:grid
        extVal += abs(sin(2*π*v/grid))
    end
    return extVal / grid
end

stepLst = [100,1000,5000]
epoch = 10

extVal = exact_sum(grid)
extValAbs = exact_sum_abs(grid)

println("Exact Value : $extVal")
println("Exact Abs Value (Z): $extValAbs")

for step in stepLst
    println("\n==== mc step : $step ====")
    
    acc = 0.0
    accSq = 0.0
    for i in 1:epoch
        mcSum = extValAbs * mcmc_sum(step)        
        acc += mcSum
        accSq += mcSum^2
    end

    

    result = acc / epoch
    
    variance = (accSq / epoch - (acc / epoch)^2)
    errVal = sqrt(max(0.0, variance) / epoch)

    println("MC result : $result")
    println("errVal : $errVal")

    if (extVal - errVal) <= result && result <= (extVal + errVal)
        println("Success :)")
    else
        println("Fail :(")
    end
end

Exact Value : 4.98699664371658e-17
Exact Abs Value (Z): 0.6364103190754791

==== mc step : 100 ====
MC result : 0.5116738965366852
errVal : 0.11833536072112544
Fail :(

==== mc step : 1000 ====
MC result : 0.12817303826180146
errVal : 0.19683956561962462
Success :)

==== mc step : 5000 ====
MC result : 0.5092555373241984
errVal : 0.12062961771469827
Fail :(
